In [1]:
import glob

In [116]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

#mark　sheetの内容=========
kumi_list=np.array(
     ["A_B","C_D","E_F","G_H","I_J","K_L","M_N","O_P","Q_R","S_T"]
)
group_list=np.array(
     ["1_2","3_4","5","6","7,8","9,10"]
)
ans_list = np.array(
     [1,2,3]
)
#===========================

def my_mark(fig_name):
    #画像の読み込み
    img_gray = cv2.GaussianBlur(cv2.imread(fig_name,0),(5,5),0) #マークシート
    template_s = cv2.imread('star.jpg',0) #位置合わせ★
    template_t = cv2.imread('tri.jpg',0) #位置合わせ▼

    #位置合わせ画像のサイズ
    ws, hs = template_s.shape[::-1]
    wt, ht = template_t.shape[::-1]

    #位置合わせ記号の座標取得
    res_s = cv2.matchTemplate(img_gray,template_s,cv2.TM_CCOEFF_NORMED)
    res_t = cv2.matchTemplate(img_gray,template_t,cv2.TM_CCOEFF_NORMED)

    #マッチング閾値（1が厳しい、0が緩い）
    threshold = 0.8

    ###座標の重複削除###
    loc_s = np.sort(np.where( res_s >= threshold) ,axis=1)
    loc_t = np.sort(np.where( res_t >= threshold) ,axis=1)

    diff_s = loc_s[0][1:]-loc_s[0][:-1]
    diff_t = loc_t[1][1:]-loc_t[1][:-1]

    loc_s = np.delete(loc_s, np.where(diff_s<hs)[0],axis=1)
    loc_t =np.delete(loc_t, np.where(diff_t<wt)[0],axis=1)
    
    
    ###画像処理###

    ### 50を閾値として2値化
    res, img = cv2.threshold(img_gray, 50, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ### 白黒反転
    img = 255 - img

    ###マーク読み取り###

    info = []#組、班
    result_tot = []#結果を入れる配列

    if len(loc_s[0])==23 and len(loc_t[1])== 18:
        for row in range(23):
            if row > 0: 
                result = []#結果を入れる配列
                #各マーク領域の画像の合計値を求める
                for theme in range(6):
                    temp = []
                    area_sum = [] # 合計値を入れる配列
                    
                    for col in range(3):
                        area_sum.append(
                            np.sum(
                                img[loc_s[0][row]:loc_s[0][row]+wt
                                    ,loc_t[1][3*theme+col]:loc_t[1][3*theme+col]+hs]
                            )
                        )
                    temp.append(area_sum > np.median(area_sum) * 3)
                
                    if len(ans_list[temp]) > 1:
                        result.append("*複*：{}".format('_'.join(map(str,ans_list[temp]))))
                    elif len(ans_list[temp]) < 1:
                        result.append("=未=")
                    else:
                        result.append(ans_list[temp][0])
                
                result_tot.append(result)
            else:
                area_sum = [] # 合計値を入れる配列
                for col in range(10):
                    area_sum.append(
                        np.sum(
                            img[loc_s[0][row]:loc_s[0][row]+wt,
                                loc_t[1][col]:loc_t[1][col]+hs]
                            )
                        )
                info.append(kumi_list[area_sum > np.median(area_sum) * 3][0])
                
                area_sum = [] # 合計値を入れる配列
                for col in range(6):
                    area_sum.append(
                        np.sum(
                            img[loc_s[0][row]:loc_s[0][row]+wt,
                                loc_t[1][col+12]:loc_t[1][col+12]+hs]
                            )
                        )
                info.append(group_list[area_sum > np.median(area_sum) * 3][0])
        
        return(info,result_tot)
    else:
        return("error")

In [128]:
col_title = [
    "単振り子,連成振動,オシロスコープ,半導体素子,電気抵抗,光の干渉",
    "単振り子,オシロスコープ,半導体素子,電気抵抗,光の干渉,連成振動",
    "単振り子,半導体素子,電気抵抗,光の干渉,連成振動,オシロスコープ",
    "単振り子,電気抵抗,光の干渉,連成振動,オシロスコープ,半導体素子",
    "単振り子,光の干渉,連成振動,オシロスコープ,半導体素子,電気抵抗"
]

with open("scan.csv", "w") as f:
    for filename in glob.glob("scans/*.jpg"):
        info, res = my_mark(filename)
    
        print("{}組_{}班".format(info[0],info[1]), file=f)
        
        print(col_title[np.where(group_list == info[1])[0][0]],file=f)
    
        for temp in res:
            print(",".join(map(str,temp)),file=f)

In [129]:
info

['A_B', '1_2']

In [130]:
res

[[1, 1, 2, 2, 1, 2],
 [2, 2, 2, 1, 2, 2],
 [2, 3, 1, 1, 1, 1],
 [1, 2, 2, 2, 2, 3],
 [3, 2, 2, 1, 1, 2],
 [2, 1, 1, 1, 1, 1],
 [3, 2, 1, 1, 1, 1],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未='],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未='],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未='],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未='],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未='],
 [2, 2, 1, 1, 3, 1],
 [1, 1, 2, 1, 1, 2],
 [3, 2, 1, 2, 1, 1],
 [2, 1, 1, 1, 1, 3],
 [1, 1, 2, 1, 3, 1],
 [2, 2, 2, 2, 2, 2],
 [3, 1, 1, 2, 1, 1],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未='],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未='],
 ['=未=', '=未=', '=未=', '=未=', '=未=', '=未=']]